## Run this first if you want NDVI/NDWI basemaps

In [1]:
import ee

ee.Authenticate(force=True)


Successfully saved authorization token.


## Vibes

In [1]:
%load_ext autoreload
%autoreload 2

from geovibes.ui import GeoVibes

vibes = GeoVibes(
    start_date="2024-01-01", end_date="2025-01-01", enable_ee=True, verbose=False
)

## Example adding ESRI LULC

In [2]:
import ee

ee.Initialize()
ee.Authenticate()
lulc = ee.ImageCollection(
    "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"
)

# Get 2024 mosaic and remap to sequential values
lulc_2024 = (
    lulc.filterDate("2024-01-01", "2024-12-31")
    .mosaic()
    .remap([1, 2, 4, 5, 7, 8, 9, 10, 11], [1, 2, 3, 4, 5, 6, 7, 8, 9])
)

# Visualization: Water, Trees, Flooded Veg, Crops, Built, Bare, Snow, Clouds, Rangeland
vis_params = {
    "min": 1,
    "max": 9,
    "palette": [
        "#1A5BAB",  # Water
        "#358221",  # Trees
        "#87D19E",  # Flooded Vegetation
        "#FFDB5C",  # Crops
        "#ED022A",  # Built Area
        "#EDE9E4",  # Bare Ground
        "#F2FAFF",  # Snow/Ice
        "#C8C8C8",  # Clouds
        "#C6AD8D",  # Rangeland
    ],
}

vibes.add_ee_layer(lulc_2024, vis_params, name="LULC 2024", opacity=0.7)

## Example adding SRTM Elevation

In [7]:
# Add elevation layer from SRTM dataset
dataset = ee.Image("USGS/SRTMGL1_003")
elevation = dataset.select("elevation")

# Add elevation layer to the map
elevation_vis_params = {
    "min": 0,
    "max": 300,
    "palette": [
        "#000080",
        "#0000FF",
        "#00FFFF",
        "#00FF00",
        "#FFFF00",
        "#FF8000",
        "#FF0000",
        "#800000",
    ],  # Blue to red gradient for elevation
}

vibes.add_ee_layer(elevation, elevation_vis_params, name="SRTM Elevation", opacity=0.7)